# Complete Erosion Simulator - All-in-One Notebook

This notebook contains everything you need to run the landscape evolution simulator with erosion analysis.

**Run cells in order from top to bottom.**

## Contents
1. Import dependencies
2. Import the landscape_evolution package
3. Generate initial terrain
4. Initialize world state with stratigraphy
5. Set up external forcing (tectonics + climate)
6. Create and run the simulator
7. Compute water routing
8. Plot comprehensive erosion analysis
9. Plot erosion rate with rivers
10. Additional visualizations

---
## 1. Import Dependencies

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors

# Configure matplotlib for inline plots
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150

print("✓ Dependencies imported successfully")

---
## 2. Import Landscape Evolution Package

In [ ]:
# Import the complete landscape evolution framework
from landscape_evolution import (
    # Core state
    WorldState,
    
    # Forcing
    TectonicUplift,
    WeatherGenerator,
    
    # Hydrology
    FlowRouter,
    
    # Simulation
    LandscapeEvolutionSimulator,
    
    # Visualization
    plot_erosion_analysis,
    plot_erosion_rate_map
)

# Import terrain generation
from landscape_evolution.terrain_generation import (
    quantum_seeded_topography,
    denormalize_elevation
)

# Import stratigraphy initialization
from landscape_evolution.initial_stratigraphy import (
    create_slope_dependent_stratigraphy
)

# Import additional visualization
from landscape_evolution.visualization import (
    plot_initial_vs_final,
    plot_erosion_deposition_maps,
    plot_river_network,
    plot_cross_section
)

print("✓ Landscape evolution package imported successfully")

---
## 3. Generate Initial Terrain

Create the initial topography using quantum-seeded terrain generation.

In [ ]:
# Parameters
N = 256  # Grid size (256x256)
pixel_scale_m = 100.0  # 100 meters per pixel
elev_range_m = (0.0, 1000.0)  # Elevation range: 0 to 1000 m

print(f"Generating terrain: {N}×{N} grid, pixel spacing = {pixel_scale_m} m")
print("This may take a moment...\n")

# Generate normalized terrain [0, 1]
z_norm, rng = quantum_seeded_topography(
    N=N,
    beta=3.1,           # Smoothness parameter
    warp_amp=0.12,      # Domain warping
    ridged_alpha=0.18,  # Ridge sharpening
    random_seed=42      # For reproducibility
)

# Convert to actual elevation in meters
surface_elev = denormalize_elevation(z_norm, elev_range_m)

print(f"✓ Terrain generated")
print(f"  Elevation range: [{surface_elev.min():.1f}, {surface_elev.max():.1f}] m")
print(f"  Mean elevation: {surface_elev.mean():.1f} m\n")

# Visualize initial terrain
plt.figure(figsize=(10, 8))
plt.imshow(surface_elev, cmap='terrain', origin='lower')
plt.colorbar(label='Elevation (m)', shrink=0.8)
plt.title('Initial Terrain', fontsize=14, fontweight='bold')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()

---
## 4. Initialize World State with Stratigraphy

Create the world state and add geological layers.

In [ ]:
# Define geological layers (top to bottom)
layer_names = [
    "Topsoil",      # Surface soil
    "Colluvium",    # Hillslope deposits
    "Saprolite",    # Weathered bedrock
    "WeatheredBR",  # Weathered bedrock rind
    "Sandstone",    # Sedimentary bedrock
    "Shale",        # Sedimentary bedrock
    "Basement"      # Crystalline basement
]

print(f"Creating world state with {len(layer_names)} layers...\n")

# Create world state
world = WorldState(
    nx=N,
    ny=N,
    pixel_scale_m=pixel_scale_m,
    layer_names=layer_names
)

# Initialize with slope-dependent stratigraphy
# (thicker on gentle slopes, thinner on steep slopes)
create_slope_dependent_stratigraphy(
    world,
    surface_elev=surface_elev,
    pixel_scale_m=pixel_scale_m,
    base_regolith_m=2.0,      # Base regolith thickness
    base_saprolite_m=5.0,     # Base saprolite thickness
    bedrock_thickness_m=100.0 # Bedrock thickness
)

print("✓ World state initialized\n")
print(world.summary())

---
## 5. Set Up External Forcing

Define tectonic uplift and climate/weather patterns.

In [ ]:
print("Setting up external forcing...\n")

# Tectonic uplift
tectonics = TectonicUplift(N, N, pixel_scale_m)
tectonics.set_uniform_uplift(1e-3)  # 1 mm/yr uniform uplift

print(f"✓ Tectonics: {tectonics}")

# Weather/climate generator
weather = WeatherGenerator(
    N, N, pixel_scale_m,
    mean_annual_precip_m=1.0,    # 1 m/yr rainfall
    wind_direction_deg=270.0,    # Wind from west
    orographic_factor=0.5        # Orographic enhancement
)

print(f"✓ Weather: {weather}\n")

---
## 6. Create and Run the Simulator

This is the main time-stepping evolution engine.

In [ ]:
print("Creating landscape evolution simulator...\n")

# Create simulator
simulator = LandscapeEvolutionSimulator(
    world=world,
    tectonics=tectonics,
    weather=weather,
    snapshot_interval=50,  # Save snapshot every 50 steps
    verbose=True           # Print progress
)

print("✓ Simulator created\n")
print("="*60)

In [ ]:
# Run simulation
total_time = 5000.0  # 5,000 years
dt = 10.0            # 10-year time steps

print(f"\nRunning simulation for {total_time:.0f} years with dt={dt} years")
print(f"Total steps: {int(total_time/dt)}")
print("This will take a few minutes...\n")
print("="*60)

# Run!
history = simulator.run(total_time=total_time, dt=dt)

print("\n" + "="*60)
print("✓ SIMULATION COMPLETE!")
print(f"  Final time: {world.time:.1f} years")
print(f"  Snapshots saved: {len(history.times)}")
print("="*60)

---
## 7. Compute Water Routing

Calculate flow directions and accumulation for river network visualization.

In [ ]:
print("Computing water routing on final surface...\n")

# Create flow router
flow_router = FlowRouter(pixel_scale_m)

# Compute flow on final surface
flow_dir, slope, flow_accum = flow_router.compute_flow(
    world.surface_elev,
    fill_depressions=False
)

print(f"✓ Flow routing complete")
print(f"  Max flow accumulation: {flow_accum.max():.0f} cells")
print(f"  Mean slope: {slope.mean():.4f} m/m\n")

---
## 8. Plot Comprehensive Erosion Analysis

Multi-panel view showing:
- Erosion map with statistics
- Histogram of erosion values
- Cross-section profile
- Erosion vs elevation scatter plot

In [ ]:
# Get cumulative erosion from simulation history
cumulative_erosion = history.get_total_erosion()

print(f"Total erosion: {cumulative_erosion.sum():.1f} m")
print(f"Mean erosion: {cumulative_erosion.mean():.2f} m")
print(f"Max erosion: {cumulative_erosion.max():.2f} m\n")

# Create comprehensive erosion analysis plot
plot_erosion_analysis(
    erosion=cumulative_erosion,
    surface_elev=world.surface_elev,
    pixel_scale_m=pixel_scale_m,
    row_for_profile=N//2  # Middle row
)

---
## 9. Plot Erosion Rate with Rivers

Shows erosion rate overlaid with the river network.

In [ ]:
# Calculate average erosion rate
erosion_rate = cumulative_erosion / total_time  # m/yr

print(f"Mean erosion rate: {erosion_rate.mean():.2e} m/yr")
print(f"Max erosion rate: {erosion_rate.max():.2e} m/yr\n")

# Plot erosion rate with river network overlay
plot_erosion_rate_map(
    erosion_rate=erosion_rate,
    pixel_scale_m=pixel_scale_m,
    flow_accum=flow_accum  # Add river overlay
)

---
## 10. Additional Visualizations

More views of the landscape evolution.

### 10.1 Initial vs Final Topography

In [ ]:
plot_initial_vs_final(history, pixel_scale_m)

### 10.2 Erosion and Deposition Maps

In [ ]:
plot_erosion_deposition_maps(history, pixel_scale_m)

### 10.3 River Network

In [ ]:
plot_river_network(
    world.surface_elev,
    flow_accum,
    pixel_scale_m,
    threshold_cells=0.01 * N * N  # 1% of domain
)

### 10.4 Cross-Section Through Stratigraphy

In [ ]:
plot_cross_section(
    world,
    row=N//2,
    vertical_exaggeration=2.0
)

---
## 11. Summary Statistics

In [ ]:
# Calculate summary statistics
initial_surface = history.surface_snapshots[0]
final_surface = history.surface_snapshots[-1]

net_change = final_surface - initial_surface
total_erosion = history.get_total_erosion()
total_deposition = history.get_total_deposition()

print("="*60)
print("SIMULATION SUMMARY")
print("="*60)
print(f"\nSimulation Time: {total_time:.0f} years")
print(f"Time Steps: {int(total_time/dt)}")
print(f"Grid Size: {N}×{N} ({N*N:,} cells)")
print(f"Pixel Scale: {pixel_scale_m} m")

print("\nELEVATION CHANGES:")
print(f"  Initial mean: {initial_surface.mean():.1f} m")
print(f"  Final mean: {final_surface.mean():.1f} m")
print(f"  Change: {(final_surface.mean() - initial_surface.mean()):.1f} m")

print("\nEROSION:")
print(f"  Total: {total_erosion.sum():.1f} m (cumulative)")
print(f"  Mean: {total_erosion.mean():.2f} m")
print(f"  Max: {total_erosion.max():.2f} m")
print(f"  Mean rate: {(total_erosion.mean()/total_time):.2e} m/yr")

print("\nDEPOSITION:")
print(f"  Total: {total_deposition.sum():.1f} m (cumulative)")
print(f"  Mean: {total_deposition.mean():.2f} m")
print(f"  Max: {total_deposition.max():.2f} m")

print("\nNET CHANGE:")
print(f"  Total lowering: {net_change[net_change < 0].sum():.1f} m")
print(f"  Total raising: {net_change[net_change > 0].sum():.1f} m")
print(f"  Net: {net_change.sum():.1f} m")

print("\nUPLIFT:")
uplift_total = tectonics.uplift_rate.mean() * total_time
print(f"  Rate: {tectonics.uplift_rate.mean():.2e} m/yr")
print(f"  Total: {uplift_total:.2f} m")

print("\n" + "="*60)

---
## 12. Export Results (Optional)

Save erosion data for external analysis.

In [ ]:
# Uncomment to save data

# # Save cumulative erosion
# np.save('erosion_cumulative.npy', cumulative_erosion)
# print("✓ Saved: erosion_cumulative.npy")

# # Save erosion rate
# np.save('erosion_rate.npy', erosion_rate)
# print("✓ Saved: erosion_rate.npy")

# # Save final surface
# np.save('surface_final.npy', world.surface_elev)
# print("✓ Saved: surface_final.npy")

# # Save as text file (slower, larger)
# # np.savetxt('erosion.txt', cumulative_erosion, fmt='%.3f')

print("Uncomment lines above to save data files")

---
## Done!

You've successfully:
- ✅ Generated initial terrain
- ✅ Initialized stratigraphy
- ✅ Run landscape evolution simulation
- ✅ Analyzed erosion patterns
- ✅ Visualized results

### Next Steps

1. **Adjust parameters**: Try different uplift rates, rainfall, time periods
2. **Longer simulations**: Increase `total_time` to 10,000 or 50,000 years
3. **Different terrain**: Change `random_seed` or terrain parameters
4. **Custom forcing**: Use `set_regional_pattern()` for non-uniform uplift
5. **Integrate your stratigraphy**: Use your `generate_stratigraphy()` function

See the documentation files for more details!